In [ ]:
!pip install transformers sentencepiece


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name = 'google/flan-t5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

...

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Test on a single data point

import time

# Sample job ad

start_time = time.time()
job_ad = "We are looking for a software developer to join our team in our downtown office in New York. Must be available to work in person 5 days a week."

# Format the prompt

prompt = f"Classify the work arrangement of the following job ad: {job_ad}"

# Tokenize and move to GPU
inputs = tokenizer(prompt, return_tensors='pt').to(device)
outputs = model.generate(**inputs, max_new_tokens=10)

stop_time = time.time()

# Decode
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print('time taken to run:', stop_time - start_time)


in person
time taken to run: 1.2899587154388428


Get data from Google Drive

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/AB_job_data_files'

import os
from google.colab import drive

# find location of different files:
file_location = {}
# Check if the folder exists
if os.path.exists(folder_path):
  os.chdir(folder_path)
  for num, f in enumerate(os.listdir()):  # List files and directories in the current folder
    file_location[f] = os.path.join(folder_path, f)

else:
  print(f"Folder not found: {folder_path}")

print(file_location)

Mounted at /content/drive
{'seniority_labelled_test_set.csv': '/content/drive/MyDrive/AB_job_data_files/seniority_labelled_test_set.csv', 'unlabelled_development_set.csv': '/content/drive/MyDrive/AB_job_data_files/unlabelled_development_set.csv', 'salary_labelled_development_set.csv': '/content/drive/MyDrive/AB_job_data_files/salary_labelled_development_set.csv', 'seniority_labelled_development_set.csv': '/content/drive/MyDrive/AB_job_data_files/seniority_labelled_development_set.csv', 'work_arrangements_test_set.csv': '/content/drive/MyDrive/AB_job_data_files/work_arrangements_test_set.csv', 'salary_labelled_test_set.csv': '/content/drive/MyDrive/AB_job_data_files/salary_labelled_test_set.csv', 'work_arrangements_development_set.csv': '/content/drive/MyDrive/AB_job_data_files/work_arrangements_development_set.csv'}


Function to extract work arrangements

In [ ]:
def work_arrang(job_ad):

    # Format the prompt
    prompt = f"Classify the work arrangement of the following job ad as one of the following: on-site, Remote, or Hybrid {job_ad}"

    # Tokenize and move to GPU
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    outputs = model.generate(**inputs, max_new_tokens=10)

    # Decode
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

Testing on development set

In [ ]:
# DEV set
import pandas as pd
work_arrangement_dev = pd.read_csv(file_location['work_arrangements_development_set.csv'])

results = 0
count = 0
for index, row in work_arrangement_dev.iterrows():
    predicted_label = work_arrang(row['job_ad'])
    if predicted_label == "on-site":
        predicted_label = "OnSite"

    true_label =  row['y_true']

    #print(work_arrang(row['job_ad']), '----', row['y_true'])
    #print(predicted_label == true_label)
    if predicted_label == true_label:
        results += 1
    count += 1

print("count:", count)
print("result:", results)
print("correctly indentified:", results/count)


count: 99
result: 53
correctly indentified: 0.5353535353535354


Testing on Test set

In [ ]:
# TEST set
import pandas as pd
work_arrangement_dev = pd.read_csv(file_location['work_arrangements_test_set.csv'])

results = 0
count = 0
for index, row in work_arrangement_dev.iterrows():
    predicted_label = work_arrang(row['job_ad'])
    if predicted_label == "on-site":
        predicted_label = "OnSite"

    true_label =  row['y_true']

    #print(work_arrang(row['job_ad']), '----', row['y_true'])
    #print(predicted_label == true_label)
    if predicted_label == true_label:
        results += 1
    count += 1

print("count:", count)
print("result:", results)
print("correctly indentified:", results/count)


count: 99
result: 45
correctly indentified: 0.45454545454545453
